<a href="https://colab.research.google.com/github/ykim71/thesis_related/blob/main/ner_pos_matching_wpreprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
from google.colab import drive

drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [18]:
%cd drive/'MyDrive'/CrowdTangle/final_data_metrics

/content/drive/MyDrive/CrowdTangle/final_data_metrics




## pre-processing

In [ ]:
# import words/phrases from NER and POS tagging
import pandas as pd

ner_pos = pd.read_excel('pos_bigrams_all_merged_NER4.xlsx')


In [ ]:
import re

def bigrams(s): # more than bigrams
    return [i for i in s if re.search(r'\s', i) ] 
def unigrams(s):
    return [i for i in s if not re.search(r'\s', i) ]
def hashtag(s):
    return [i for i in s if re.search(r'#', i) ]

def contain_the(s): # bigrams that make sense with 'the'
  to_remove = [' #', '# ', '.','\'s', "’s", " \'", "\'", "\'S", '“', '"', '’', ]
  p = re.compile('|'.join(map(re.escape, to_remove))) # escape to handle metachars
  s = [p.sub('', s) for s in s] # remove hashtags

  matchers = ['The ', 'the ']
  items = [i for i in s if any(xs in i for xs in matchers)]
  temp = [i for i in items if i.count(' ')==1]
  return list(set(temp))


In [ ]:
# for Republican words/phrases

ner_rep_only = ner_pos.loc[(ner_pos['rep'] == 1)]

temp = ner_rep_only['text'].values.tolist()
ner_rep_only_list = list(set(temp))
print(len(ner_rep_only_list))

# bi-grams
rep_bi = list(set(bigrams(ner_rep_only_list)))

import re

to_remove = [' #', '# ', '.','\'s', "’s", " \'", "\'", "\'S", '“', '"', '’', ' Timeline', ' Photo', 'the ', 'The ']

p = re.compile('|'.join(map(re.escape, to_remove))) # escape to handle metachars

rep_bi = [p.sub('', s) for s in rep_bi] # remove hashtags

rep_bi = list(set(bigrams(rep_bi))) # select only bi-grams (hashtags will be added here)

# uni-grams

rep_uni = list(set(unigrams(ner_rep_only_list)))
print(len(rep_uni))
import re

to_remove = ['#', '.', '\'s', "’s", " \'", "\'", "\'S", '“', '"', '’']

p = re.compile('|'.join(map(re.escape, to_remove))) # escape to handle metachars

rep_uni = [p.sub('', s) for s in rep_uni] # remove some characters
rep_uni = list(set(rep_uni))
print(len(rep_uni))


# hashtags

import re

rep_hash = list(set(hashtag(ner_rep_only_list)))

to_remove = [' #', '# ', '#']
p = re.compile('|'.join(map(re.escape, to_remove))) # escape to handle metachars

rep_hash = [p.sub('', s) for s in rep_hash] # remove hashtags
rep_hash_non = list(set(rep_hash))

rep_hash = ['#' + s for s in rep_hash]
rep_hash = list(set(rep_hash))

# bigrams with the

rep_bi_the = contain_the(list(set(bigrams(ner_rep_only_list)))) 
print(len(rep_bi_the))

2712
185
181
1


In [ ]:
rep_bi_hashtags = rep_bi + rep_hash + rep_bi_the
rep_uni_nonhashtags = rep_uni + rep_hash_non


In [ ]:
# for Democratic words/phrases

ner_dem_only = ner_pos.loc[(ner_pos['dem'] == 1)]

temp = ner_dem_only['text'].values.tolist()
ner_dem_only_list = list(set(temp))
print(len(ner_dem_only_list))

# bi-grams
dem_bi = list(set(bigrams(ner_dem_only_list)))

import re

to_remove = [' #', '# ', '.','\'s', "’s", " \'", "\'", "\'S", '“', '"', '’', ' Timeline', ' Photo', 'the ', 'The ']

p = re.compile('|'.join(map(re.escape, to_remove))) # escape to handle metachars

dem_bi = [p.sub('', s) for s in dem_bi] # remove hashtags

dem_bi = list(set(bigrams(dem_bi))) # select only bi-grams (hashtags will be added here)


# uni-grams

dem_uni = list(set(unigrams(ner_dem_only_list)))
print(len(dem_uni))
import re

to_remove = ['#', '.', '\'s', "’s", " \'", "\'", "\'S", '“', '"', '’']

p = re.compile('|'.join(map(re.escape, to_remove))) # escape to handle metachars

dem_uni = [p.sub('', s) for s in dem_uni] # remove some characters
dem_uni = list(set(dem_uni))
print(len(dem_uni))

# hashtags

import re

dem_hash = list(set(hashtag(ner_dem_only_list)))

to_remove = [' #', '# ', '#']
p = re.compile('|'.join(map(re.escape, to_remove))) # escape to handle metachars

dem_hash = [p.sub('', s) for s in dem_hash] # remove hashtags
dem_hash_non = list(set(dem_hash))

dem_hash = ['#' + s for s in dem_hash]
dem_hash = list(set(dem_hash))


# bigrams with the

dem_bi_the = contain_the(list(set(bigrams(ner_dem_only_list)))) 


2533
175
173


In [ ]:

dem_bi_hashtags = dem_bi + dem_hash + dem_bi_the
dem_uni_nonhashtags = dem_uni + dem_hash_non


In [ ]:
# make lowercases for matching

temp_rep_lower_bi = list( set([x.lower() for x in rep_bi_hashtags] ))
temp_dem_lower_bi = list( set([x.lower() for x in dem_bi_hashtags] ))

temp_rep_lower_uni = list( set([x.lower() for x in rep_uni_nonhashtags]) )
temp_dem_lower_uni = list( set([x.lower() for x in dem_uni_nonhashtags]) )

# temp_rep_lower_bi_the = list( set([x.lower() for x in rep_bi_the] ))
# temp_dem_lower_bi_the = list( set([x.lower() for x in dem_bi_the] ))

In [ ]:
print(len(temp_rep_lower_bi))
print(len(temp_dem_lower_bi))

print(len(temp_rep_lower_uni))
print(len(temp_dem_lower_uni))


2133
2059
245
236


## matching sample

In [19]:
# for entire posts
import re

def basic_sanitize(text):
    
    text = re.sub('’s',' s', text) #&
    text = re.sub('\'s',' s', text) #&
    text = re.sub('’S',' S', text) #&
    text = re.sub('\'S',' S', text) #&

    text = re.sub(r'[^\w\s#&-]','', text) #&
    
    return text

In [20]:
import pickle
import pandas as pd
ner = pd.read_excel("pos_bigrams_all_merged_NER5_final.xlsx")
pos = pd.read_excel("pos_both_all_no_san_c300_merge_trim.xlsx")


In [21]:
ner_y = ner[ner['NER']=='Y']

In [22]:
ner_y['text_san'] = ner_y['text'].apply(basic_sanitize)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [23]:
pos['text_san'] = pos['text'].apply(basic_sanitize)

In [34]:
ner_y_temp = ner_y[['text','text_san', 'count_sum', 'dem', 'rep']]

In [24]:
ner_y[ (ner_y['dem']==1) | (ner_y['rep']==1)].sample(10)

,id,pos_junky,text,count_dem,count_sum,politician,dem,rep,media,Unnamed: 9,NER,text_san
67310,67351,3,Greg Kelly,NaN,NaN,0,0,1,1,done,Y,Greg Kelly
28895,28909,3,JB Pritzker,548.0,875.0,1,1,0,0,done,Y,JB Pritzker
72850,72895,3,TeaParty,NaN,NaN,0,0,1,0,done,Y,TeaParty
71806,71851,3,Romney/Ryan,NaN,NaN,1,0,1,0,done,Y,RomneyRyan
64148,64182,3,Barack,NaN,NaN,1,1,0,0,done,Y,Barack
73099,73144,3,The Carter Center,NaN,NaN,0,1,0,0,done,Y,The Carter Center
11721,11724,3,Conor Lamb,1120.0,1370.0,1,1,0,0,done,Y,Conor Lamb
49270,49289,3,Richard Neal,860.0,1066.0,1,1,0,0,done,Y,Richard Neal
50413,50432,3,Sam Clovis,362.0,420.0,1,0,1,0,done,Y,Sam Clovis
65316,65351,3,ClintonScandals,NaN,NaN,1,1,0,0,done,Y,ClintonScandals


In [35]:
pos_y_temp = pos[['text','text_san', 'count_sum', 'dem', 'rep']]

In [25]:
pos[ (pos['dem']==1) | (pos['rep']==1)].sample(2)

,Unnamed: 0,Unnamed: 0.1,text,text_san,check,rep,dem,count_sum,type,count_sum_clean
64792,59987,6076920,Sen. Kyrsten,Sen Kyrsten,1,0,1,306,bigram,306.0
72624,66006,6690443,Tom Bossert,Tom Bossert,1,1,0,313,bigram,316.0


In [45]:
all_temp = pd.concat([pos_y_temp, ner_y_temp ])

In [51]:
all_temp_dem = all_temp[ (all_temp['dem'] >=1) ]
all_temp_rep = all_temp[ (all_temp['rep'] >=1) ]

In [57]:
all_temp_dem = all_temp_dem.drop_duplicates(subset='text_san', keep="last")
all_temp_rep = all_temp_rep.drop_duplicates(subset='text_san', keep="last")

In [60]:
all_temp_dem

,text,text_san,count_sum,dem,rep
4,@AdamParkhomenko,AdamParkhomenko,343.0,1,0
5,@Alyssa_Milano,Alyssa_Milano,375.0,1,0
20,@brianschatz,brianschatz,341.0,1,0
31,@CoriBush,CoriBush,327.0,1,0
40,@donwinslow,donwinslow,305.0,1,0
...,...,...,...,...,...
75440,Whitmer,Whitmer,NaN,1,0
75486,William J. Barber II,William J Barber II,NaN,1,0
75487,William Lacy Clay,William Lacy Clay,NaN,1,0
75495,Willie Brown,Willie Brown,NaN,1,0


In [56]:
len(all_temp_rep)

3703

In [ ]:
all_temp2_dem = all_temp2[all_temp2]

In [42]:
all_temp2 = all_temp2.drop_duplicates()

In [44]:
all_temp2.to_excel('all_temp2.xlsx')

In [41]:
len(list(set(all_temp2.text_san.values.tolist())))

5472

In [ ]:
# # for hashtags 
# import re

# def hash_sanitize(text):
#   if re.search(r'#', text):
#     to_remove = [' #', '# ', '#']
#     p = re.compile('|'.join(map(re.escape, to_remove)))
#     text = '#' + text
    
#     return text


# to_remove = [' #', '# ', '#']
# p = re.compile('|'.join(map(re.escape, to_remove))) # escape to handle metachars

# dem_hash = [p.sub('', s) for s in dem_hash] # remove hashtags
# dem_hash_non = list(set(dem_hash))

# dem_hash = ['#' + s for s in dem_hash]
# dem_hash = list(set(dem_hash))

# print(len(dem_hash))
# print(len(dem_hash_non))

In [26]:
import re

def bigrams(s): # more than bigrams
    return [i for i in s if re.search(r'\s', i) ] 
def unigrams(s):
    return [i for i in s if not re.search(r'\s', i) ]
def hashtag(s):
    return [i for i in s if re.search(r'#', i) ]


In [27]:
# ner

ner_dem_only = ner_y.loc[(ner_y['dem'] >= 1)]

temp = ner_dem_only['text_san'].values.tolist()
#temp = ner_dem_only['text'].values.tolist()
ner_dem_only_list = list(set(temp))
print(len(ner_dem_only_list))

dem_bi_ner = list(set(bigrams(ner_dem_only_list)))
print(len(dem_bi_ner))

dem_uni_ner = list(set(unigrams(ner_dem_only_list)))
print(len(dem_uni_ner))

## pos
pos_dem_only = pos.loc[(pos['dem'] >= 1)]

temp = pos_dem_only['text_san'].values.tolist()
#temp = pos_dem_only['text'].values.tolist()
pos_dem_only_list = list(set(temp))
print(len(pos_dem_only_list))

dem_bi_pos = list(set(bigrams(pos_dem_only_list)))
print(len(dem_bi_pos))

dem_uni_pos = list(set(unigrams(pos_dem_only_list)))
print(len(dem_uni_pos))



1439
1277
162
2477
2147
330


In [28]:


# hash - ner
import re

dem_hash = list(set(hashtag(ner_dem_only_list + pos_dem_only_list)))


In [30]:
dem_hash[1:3]

['# AOC', '# Democrat']

In [31]:

to_remove = [' #', '# ', '#']
p = re.compile('|'.join(map(re.escape, to_remove))) # escape to handle metachars

dem_hash = [p.sub('', s) for s in dem_hash] # remove hashtags
dem_hash_non = list(set(dem_hash))

dem_hash = ['#' + s for s in dem_hash]
dem_hash = list(set(dem_hash))

print(len(dem_hash))
print(len(dem_hash_non))

75
75


In [33]:
print(dem_hash_non)


['StillSanders', 'NeverHillary', 'WhichHillary', 'FeelTheBern', 'BernieOrBust', 'Bernie2016', 'OurRevolution', 'BernieSanders', 'AOC', 'Bernie2020', 'VoteBlueNoMatterWho', 'democrats', 'Labor4Bernie', 'Clinton', 'Sanders2016', 'neverhillary', 'bernieorbust', 'Liberal', 'HillaryForPrison', 'StrongerTogether', 'cnn', 'SocialismSucks', 'ThanksObama', 'Antifa', 'obama', 'CrookedHillary', 'DemDebate', 'DemConvention', 'Progressives', 'Hillary', 'socialists', 'ObamaGate', 'NEVERHILLARY', 'BidenHarris', 'BernieSanders2020', 'PresidentTrump', 'DemExit', 'Obamacare', 'draintheswamp', 'JoeBiden', 'ACA', 'BidenHarris2020', 'Sanders', 'FEELTHEBERN', 'Biden2020', 'Hillary2016', 'DrainTheSwamp', 'ImWithHer', 'DNC', 'socialism', 'uniteblue', 'Obamagate', 'feelthebern', 'Bernie', 'Democrats', 'democrat', 'Democrat', 'Women4Bernie', 'Obama', 'BernieorBust', 'CNN', 'voteblue', 'BarackObama', 'FeeltheBern', 'berniesanders', 'UniteBlue', 'Biden', 'VivaBernie', 'Socialism', 'HillaryClinton', 'OBAMAGATE', '

In [32]:
print(dem_hash)


['#berniesanders', '#BrandNewCongress', '#FeelTheBern', '#Democrat', '#bernieorbust', '#BernieOrBust', '#Sanders', '#FeeltheBern', '#Clinton', '#JoeBiden', '#VoteBlueNoMatterWho', '#BidenHarris2020', '#UniteBlue', '#Bernie2020', '#Hillary', '#Biden2020', '#Obamagate', '#uniteblue', '#NEVERHILLARY', '#Labor4Bernie', '#BidenHarris', '#OurRevolution', '#DNC', '#voteblue', '#BarackObama', '#obama', '#socialists', '#Women4Bernie', '#ObamaGate', '#draintheswamp', '#ImWithHer', '#democrat', '#neverhillary', '#SocialismSucks', '#VivaBernie', '#DemExit', '#HillaryClinton', '#Biden', '#BernieorBust', '#BernieSanders2020', '#StrongerTogether', '#Progressives', '#feelthebern', '#Democrats', '#WhichHillary', '#cnn', '#NeverBiden', '#AOC', '#Bernie', '#FEELTHEBERN', '#BernieSanders', '#HillaryForPrison', '#CrookedHillary', '#NeverHillary', '#PresidentTrump', '#VoteBlue', '#Obama', '#Socialism', '#Hillary2016', '#OBAMAGATE', '#Sanders2016', '#socialism', '#DrainTheSwamp', '#CNN', '#DemDebate', '#Stil

In [20]:
# ner

ner_rep_only = ner_y.loc[(ner_y['rep'] >= 1)]

#temp = ner_rep_only['text_san'].values.tolist()
temp = ner_rep_only['text'].values.tolist()
ner_rep_only_list = list(set(temp))
print(len(ner_rep_only_list))

rep_bi_ner = list(set(bigrams(ner_rep_only_list)))
print(len(rep_bi_ner))

rep_uni_ner = list(set(unigrams(ner_rep_only_list)))
print(len(rep_uni_ner))

## pos
pos_rep_only = pos.loc[(pos['rep'] >= 1)]

#temp = pos_rep_only['text_san'].values.tolist()
temp = pos_rep_only['text'].values.tolist()
pos_rep_only_list = list(set(temp))
print(len(pos_rep_only_list))

rep_bi_pos = list(set(bigrams(pos_rep_only_list)))
print(len(rep_bi_pos))

rep_uni_pos = list(set(unigrams(pos_rep_only_list)))
print(len(rep_uni_pos))


# hash - ner
import re

rep_hash = list(set(hashtag(ner_rep_only_list + pos_rep_only_list)))

to_remove = [' #', '# ', '#']
p = re.compile('|'.join(map(re.escape, to_remove))) # escape to handle metachars

rep_hash = [p.sub('', s) for s in rep_hash] # remove hashtags
rep_hash_non = list(set(rep_hash))

rep_hash = ['#' + s for s in rep_hash]
rep_hash = list(set(rep_hash))

print(len(rep_hash))
print(len(rep_hash_non))


1659
1478
181
2043
1699
344
85
85


In [21]:
# text ver.
dem_bi = list(set(dem_bi_ner + dem_bi_pos + dem_hash))
print(len(dem_bi))

dem_uni = list(set(dem_uni_ner + dem_uni_pos + dem_hash_non))
print(len(dem_uni))

rep_bi = list(set(rep_bi_ner + rep_bi_pos + rep_hash))
print(len(rep_bi))

rep_uni = list(set(rep_uni_ner + rep_uni_pos + rep_hash_non))
print(len(rep_uni))

2862
386
2448
404


In [25]:

df1_temp = pd.DataFrame({'text':dem_bi})
#df1.to_csv('dem_bi.csv')
df2_temp = pd.DataFrame({'text':dem_uni})
#df2.to_csv('dem_uni.csv')

df3_temp = pd.DataFrame({'text':rep_bi})
#df3.to_csv('rep_bi.csv')
df4_temp = pd.DataFrame({'text':rep_uni})
#df4.to_csv('rep_uni.csv')

df_dem_temp = pd.concat([df1_temp, df2_temp])
df_rep_temp = pd.concat([df3_temp, df4_temp])

# df_dem_temp.to_csv('dem_entities_org.csv')
# df_rep_temp.to_csv('rep_entities_org.csv')


In [26]:
df_dem_temp['text_san'] = df_dem_temp['text'].apply(basic_sanitize)
df_rep_temp['text_san'] = df_rep_temp['text'].apply(basic_sanitize)

In [28]:
df_dem_temp.drop_duplicates(subset='text_san', keep="last")
df_rep_temp.drop_duplicates(subset='text_san', keep="last")

,text,text_san
2,Benny Johnson,Benny Johnson
3,Calls Trump,Calls Trump
4,Rudolph Giuliani,Rudolph Giuliani
5,Donald TrumpDonald,Donald TrumpDonald
6,Alfonzo Rachel,Alfonzo Rachel
...,...,...
399,Pence,Pence
400,WomenForTrump,WomenForTrump
401,BigGovSucks,BigGovSucks
402,N.R.A.,NRA


In [31]:

df_dem_temp.to_excel('dem_entities_org.xlsx', encoding='utf-8-sig')
df_rep_temp.to_excel('rep_entities_org.xlsx', encoding='utf-8-sig')


In [14]:
# text_san ver.
dem_bi = list(set(dem_bi_ner + dem_bi_pos + dem_hash))
print(len(dem_bi))

dem_uni = list(set(dem_uni_ner + dem_uni_pos + dem_hash_non))
print(len(dem_uni))

rep_bi = list(set(rep_bi_ner + rep_bi_pos + rep_hash))
print(len(rep_bi))

rep_uni = list(set(rep_uni_ner + rep_uni_pos + rep_hash_non))
print(len(rep_uni))

2831
372
2392
392


In [18]:

df1 = pd.DataFrame({'dem_entities':dem_bi})
#df1.to_csv('dem_bi.csv')
df2 = pd.DataFrame({'dem_entities':dem_uni})
#df2.to_csv('dem_uni.csv')

df3 = pd.DataFrame({'rep_entities':rep_bi})
#df3.to_csv('rep_bi.csv')
df4 = pd.DataFrame({'rep_entities':rep_uni})
#df4.to_csv('rep_uni.csv')

df_dem = pd.concat([df1, df2])
df_rep = pd.concat([df3, df4])

# df_dem.to_csv('dem_entities.csv')
# df_rep.to_csv('rep_entities.csv')


In [ ]:
import pickle
import pandas as pd

temp = pd.read_csv("r_pages_st_sample_lower.csv")

In [ ]:
temp['text_san'] = temp['text'].apply(basic_sanitize)

In [ ]:

temp['dem_bi'] = temp['text_san'].apply(lambda x: [i for i in dem_bi if i in x])
temp['dem_uni'] = temp['text_san'].apply(lambda x: set.intersection(set(x.split(' ')), dem_uni))

temp['rep_bi'] = temp['text_san'].apply(lambda x: [i for i in rep_bi if i in x])
temp['rep_uni'] = temp['text_san'].apply(lambda x: set.intersection(set(x.split(' ')), rep_uni))


In [ ]:
temp.to_csv('r_pages_st_sample_lower2.csv')

In [ ]:
# from google.colab import data_table
# data_table.enable_dataframe_formatter()

# all_pages_sample_1k[all_pages_sample_1k['text_san'].str.contains("#")]


## matching

### for Republican posts

In [ ]:
import pickle

with open('r_pages_ner_pos.pickle', 'rb') as handle:
    r_pages_st = pickle.load(handle)

In [ ]:
r_pages_st['text_san'] = r_pages_st['text'].apply(basic_sanitize)


In [ ]:
r_pages_st['text_san'].sample(3)

6830247    That didnt take very long Jon Gruden Makes His...
1982195     Terrorism Deaths Quadruple Under Obama - Brei...
3697005    JUST IN Fox News Makes Major Impeachment Move ...
Name: text_san, dtype: object

In [ ]:
print(len(dem_bi))
print(len(dem_uni))

print(len(rep_bi))
print(len(rep_uni))

2827
372
2390
392


In [ ]:
# matching

r_pages_st['dem_bi'] = r_pages_st['text_san'].apply(lambda x: [i for i in dem_bi if i in x])
r_pages_st['dem_uni'] = r_pages_st['text_san'].apply(lambda x: set.intersection(set(x.split(' ')), dem_uni))


In [ ]:

r_pages_st['rep_bi'] = r_pages_st['text_san'].apply(lambda x: [i for i in rep_bi if i in x])
r_pages_st['rep_uni'] = r_pages_st['text_san'].apply(lambda x: set.intersection(set(x.split(' ')), rep_uni))


In [ ]:
# saving results

r_pages_st6 = r_pages_st[['quarter','insult','sentistr','dem_bi','dem_uni','rep_bi','rep_uni']]

import pickle

with open('r_pages_ner_pos_final.pickle', 'wb') as handle:
    pickle.dump(r_pages_st6, handle, protocol=pickle.HIGHEST_PROTOCOL)



In [ ]:
# manual checking

r_pages_st_sample = r_pages_st.sample(1000)
r_pages_st_sample.to_csv('r_pages_st_sample_lower.csv', encoding='utf-8-sig')


### for Democratic posts

In [ ]:
import pickle

with open('d_pages_ner_pos.pickle', 'rb') as handle:
    d_pages_st = pickle.load(handle)

In [ ]:
d_pages_st['text_san'] = d_pages_st['text'].apply(basic_sanitize)


In [ ]:
d_pages_st['text_san'].sample(3)

3078922    FIVE Indiana GOP Congresspeople are among thos...
914044     NOTE Same recipe tomorrow and the next day and...
2216804    These are the 5 things you need to know about ...
Name: text_san, dtype: object

In [ ]:

d_pages_st['dem_bi'] = d_pages_st['text_san'].apply(lambda x: [i for i in dem_bi if i in x])
d_pages_st['dem_uni'] = d_pages_st['text_san'].apply(lambda x: set.intersection(set(x.split(' ')), dem_uni))


In [ ]:
d_pages_st['rep_bi'] = d_pages_st['text_san'].apply(lambda x: [i for i in rep_bi if i in x])
d_pages_st['rep_uni'] = d_pages_st['text_san'].apply(lambda x: set.intersection(set(x.split(' ')), rep_uni))


In [ ]:
# saving results

d_pages_st6 = d_pages_st[['quarter','insult','sentistr','dem_bi','dem_uni','rep_bi','rep_uni']]

import pickle

with open('d_pages_ner_pos_final.pickle', 'wb') as handle:
    pickle.dump(d_pages_st6, handle, protocol=pickle.HIGHEST_PROTOCOL)



In [ ]:
# manual checking

d_pages_st_sample = d_pages_st.sample(1000)
d_pages_st_sample.to_csv('d_pages_st_sample_lower.csv', encoding='utf-8-sig')


## merge results from both leanings

In [ ]:
import pickle

with open('r_pages_ner_pos_final.pickle', 'rb') as handle:
    r_pages_st = pickle.load(handle)

In [ ]:

with open('d_pages_ner_pos_final.pickle', 'rb') as handle:
    d_pages_st = pickle.load(handle)

In [ ]:
r_pages_st['leanings'] = "republican"
d_pages_st['leanings'] = "democratic"

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

d_pages_st[['sentistr_std', 'insult_std']] = scaler.fit_transform(d_pages_st[['sentistr', 'insult']])

r_pages_st[['sentistr_std', 'insult_std']] = scaler.fit_transform(r_pages_st[['sentistr', 'insult']])


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/extmath.py:991: RuntimeWarning: invalid value encountered in subtract
  temp = X - T
/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_data.py:993: RuntimeWarning: invalid value encountered in subtract
  X -= self.mean_
/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/extmath.py:991: RuntimeWarning: invalid value encountered in subtract
  temp = X - T
/usr/local/lib/python3.7/dist-package

In [ ]:

import numpy as np

d_pages_st['dem_bi_count'] = d_pages_st['dem_bi'].str.len()
d_pages_st['dem_uni_count'] = d_pages_st['dem_uni'].str.len()

d_pages_st['rep_bi_count'] = d_pages_st['rep_bi'].str.len()
d_pages_st['rep_uni_count'] = d_pages_st['rep_uni'].str.len()


r_pages_st['dem_bi_count'] = r_pages_st['dem_bi'].str.len()
r_pages_st['dem_uni_count'] = r_pages_st['dem_uni'].str.len()

r_pages_st['rep_bi_count'] = r_pages_st['rep_bi'].str.len()
r_pages_st['rep_uni_count'] = r_pages_st['rep_uni'].str.len()

In [ ]:
# both included
import numpy as np

d_pages_st['out_group_np'] = np.where(( (d_pages_st.rep_bi_count >= 1) | (d_pages_st.rep_uni_count >= 1) ), 'mentioned', 'non_mentioned')
d_pages_st['in_group_np'] = np.where(( (d_pages_st.dem_bi_count >= 1) | (d_pages_st.dem_uni_count >= 1) ), 'mentioned', 'non_mentioned')
d_pages_st['no_entity_np'] = np.where(((d_pages_st.in_group_np == 'non_mentioned') & (d_pages_st.out_group_np == 'non_mentioned')), 'none', 'mentioned')
d_pages_st['both_entity'] = np.where(((d_pages_st.in_group_np == 'mentioned') & (d_pages_st.out_group_np == 'mentioned')), 'both', 'not_both')


In [ ]:
import numpy as np

r_pages_st['in_group_np'] = np.where(( (r_pages_st.rep_bi_count >= 1) | (r_pages_st.rep_uni_count >= 1) ), 'mentioned', 'non_mentioned')
r_pages_st['out_group_np'] = np.where(( (r_pages_st.dem_bi_count >= 1) | (r_pages_st.dem_uni_count >= 1) ), 'mentioned', 'non_mentioned')
r_pages_st['no_entity_np'] = np.where(((r_pages_st.in_group_np == 'non_mentioned') & (r_pages_st.out_group_np == 'non_mentioned')), 'none', 'mentioned')
r_pages_st['both_entity'] = np.where(((r_pages_st.in_group_np == 'mentioned') & (r_pages_st.out_group_np == 'mentioned')), 'both', 'not_both')


In [ ]:

print('in_group_np')
print(d_pages_st['in_group_np'].value_counts())

print('out_group_np')
print(d_pages_st['out_group_np'].value_counts())

print('no_entity_np')
print(d_pages_st['no_entity_np'].value_counts())


print('in_group_np')
print(r_pages_st['in_group_np'].value_counts())

print('out_group_np')
print(r_pages_st['out_group_np'].value_counts())

print('no_entity_np')
print(r_pages_st['no_entity_np'].value_counts())

in_group_np
non_mentioned    4388139
mentioned        2421885
Name: in_group_np, dtype: int64
out_group_np
non_mentioned    3651822
mentioned        3158202
Name: out_group_np, dtype: int64
no_entity_np
mentioned    4283905
none         2526119
Name: no_entity_np, dtype: int64
in_group_np
non_mentioned    4351676
mentioned        2998962
Name: in_group_np, dtype: int64
out_group_np
non_mentioned    4685220
mentioned        2665418
Name: out_group_np, dtype: int64
no_entity_np
mentioned    4264846
none         3085792
Name: no_entity_np, dtype: int64


In [ ]:
# in-, out- only

d_pages_st['out_group_only_np'] = np.where(( ( (d_pages_st.rep_bi_count >= 1) | (d_pages_st.rep_uni_count >= 1)) & ( (d_pages_st.dem_bi_count == 0) & (d_pages_st.dem_uni_count == 0) ) ), 'mentioned', 'non_mentioned')
d_pages_st['in_group_only_np'] = np.where(( ( (d_pages_st.dem_bi_count >= 1) | (d_pages_st.dem_uni_count >= 1) ) & ( ( d_pages_st.rep_bi_count == 0) & (d_pages_st.rep_uni_count == 0) ) ), 'mentioned', 'non_mentioned')

# in-, out- only

r_pages_st['in_group_only_np'] = np.where(( ( (r_pages_st.rep_bi_count >= 1) | (r_pages_st.rep_uni_count >= 1)) & ( (r_pages_st.dem_bi_count == 0) & (r_pages_st.dem_uni_count == 0) ) ), 'mentioned', 'non_mentioned')
r_pages_st['out_group_only_np'] = np.where(( ( (r_pages_st.dem_bi_count >= 1) | (r_pages_st.dem_uni_count >= 1) ) & ( ( r_pages_st.rep_bi_count == 0) & (r_pages_st.rep_uni_count == 0) ) ), 'mentioned', 'non_mentioned')


In [ ]:

print('d_pages_st: out_group_only_np')
print(d_pages_st['out_group_only_np'].value_counts())

print('d_pages_st: in_group_only_np')
print(d_pages_st['in_group_only_np'].value_counts())


print('r_pages_st: in_group_only_np')
print(r_pages_st['in_group_only_np'].value_counts())

print('r_pages_st: out_group_only_np')
print(r_pages_st['out_group_only_np'].value_counts())



d_pages_st: out_group_only_np
non_mentioned    4948004
mentioned        1862020
Name: out_group_only_np, dtype: int64
d_pages_st: in_group_only_np
non_mentioned    5684321
mentioned        1125703
Name: in_group_only_np, dtype: int64
r_pages_st: in_group_only_np
non_mentioned    5751210
mentioned        1599428
Name: in_group_only_np, dtype: int64
r_pages_st: out_group_only_np
non_mentioned    6084754
mentioned        1265884
Name: out_group_only_np, dtype: int64


In [ ]:
d_pages_st.columns

Index(['quarter', 'insult', 'sentistr', 'dem_bi', 'dem_uni', 'rep_bi',
       'rep_uni', 'leanings', 'sentistr_std', 'insult_std', 'dem_bi_count',
       'dem_uni_count', 'rep_bi_count', 'rep_uni_count', 'out_group_np',
       'in_group_np', 'no_entity_np', 'both_entity'],
      dtype='object')

In [ ]:
r_pages_st.columns

Index(['quarter', 'insult', 'sentistr', 'dem_bi', 'dem_uni', 'rep_bi',
       'rep_uni', 'leanings', 'sentistr_std', 'insult_std', 'dem_bi_count',
       'dem_uni_count', 'rep_bi_count', 'rep_uni_count', 'in_group_np',
       'out_group_np', 'no_entity_np', 'both_entity'],
      dtype='object')

In [ ]:
import pandas as pd

pages_df_st = pd.concat([r_pages_st, d_pages_st], ignore_index=True, sort=False)


In [ ]:
# aggregated by in-, out- only

temp  = pages_df_st.groupby(['quarter', 'out_group_only_np'])['sentistr_std'].mean().reset_index()
temp.to_csv('pos_ner_basic/all_np_out_group_only_senti_std.csv', encoding='utf-8-sig')

temp  = pages_df_st.groupby(['quarter', 'in_group_only_np'])['sentistr_std'].mean().reset_index()
temp.to_csv('pos_ner_basic/all_np_in_group_only_senti_std.csv', encoding='utf-8-sig')

temp  = pages_df_st.groupby(['quarter', 'out_group_only_np'])['insult_std'].mean().reset_index()
temp.to_csv('pos_ner_basic/all_np_out_group_only_insult_std.csv', encoding='utf-8-sig')

temp  = pages_df_st.groupby(['quarter', 'in_group_only_np'])['insult_std'].mean().reset_index()
temp.to_csv('pos_ner_basic/all_np_in_group_only_insult_std.csv', encoding='utf-8-sig')


temp  = pages_df_st.groupby(['quarter', 'out_group_only_np', 'leanings'])['sentistr_std'].mean().reset_index()
temp.to_csv('pos_ner_basic/leanings_np_out_group_only_senti_std.csv', encoding='utf-8-sig')

temp  = pages_df_st.groupby(['quarter', 'in_group_only_np', 'leanings'])['sentistr_std'].mean().reset_index()
temp.to_csv('pos_ner_basic/leanings_np_in_group_only_senti_std.csv', encoding='utf-8-sig')

temp  = pages_df_st.groupby(['quarter', 'out_group_only_np', 'leanings'])['insult_std'].mean().reset_index()
temp.to_csv('pos_ner_basic/leanings_np_out_group_only_insult_std.csv', encoding='utf-8-sig')

temp  = pages_df_st.groupby(['quarter', 'in_group_only_np', 'leanings'])['insult_std'].mean().reset_index()
temp.to_csv('pos_ner_basic/leanings_np_in_group_only_insult_std.csv', encoding='utf-8-sig')


In [ ]:
# aggregated by all posts

temp  = pages_df_st.groupby(['quarter', 'out_group_np'])['sentistr_std'].mean().reset_index()
temp.to_csv('pos_ner_basic/all_np_out_group_senti_std.csv', encoding='utf-8-sig')


temp  = pages_df_st.groupby(['quarter', 'in_group_np'])['sentistr_std'].mean().reset_index()
temp.to_csv('pos_ner_basic/all_np_in_group_senti_std.csv', encoding='utf-8-sig')


temp  = pages_df_st.groupby(['quarter', 'no_entity_np'])['sentistr_std'].mean().reset_index()
temp.to_csv('pos_ner_basic/all_np_no_entities_senti_std.csv', encoding='utf-8-sig')

temp  = pages_df_st.groupby(['quarter', 'both_entity'])['sentistr_std'].mean().reset_index()
temp.to_csv('pos_ner_basic/all_np_both_entities_senti_std.csv', encoding='utf-8-sig')


temp  = pages_df_st.groupby(['quarter', 'out_group_np'])['insult_std'].mean().reset_index()
temp.to_csv('pos_ner_basic/all_np_out_group_insult_std.csv', encoding='utf-8-sig')


temp  = pages_df_st.groupby(['quarter', 'in_group_np'])['insult_std'].mean().reset_index()
temp.to_csv('pos_ner_basic/all_np_in_group_insult_std.csv', encoding='utf-8-sig')


temp  = pages_df_st.groupby(['quarter', 'no_entity_np'])['insult_std'].mean().reset_index()
temp.to_csv('pos_ner_basic/all_np_no_entities_insult_std.csv', encoding='utf-8-sig')

temp  = pages_df_st.groupby(['quarter', 'both_entity'])['insult_std'].mean().reset_index()
temp.to_csv('pos_ner_basic/all_np_both_entities_insult_std.csv', encoding='utf-8-sig')


In [ ]:
# aggregated by all posts + by leanings

temp  = pages_df_st.groupby(['quarter', 'out_group_np', 'leanings'])['sentistr_std'].mean().reset_index()
temp.to_csv('pos_ner_basic/leanings_np_out_group_senti_std.csv', encoding='utf-8-sig')


temp  = pages_df_st.groupby(['quarter', 'in_group_np', 'leanings'])['sentistr_std'].mean().reset_index()
temp.to_csv('pos_ner_basic/leanings_np_in_group_senti_std.csv', encoding='utf-8-sig')


temp  = pages_df_st.groupby(['quarter', 'no_entity_np', 'leanings'])['sentistr_std'].mean().reset_index()
temp.to_csv('pos_ner_basic/leanings_np_no_entities_senti_std.csv', encoding='utf-8-sig')


temp  = pages_df_st.groupby(['quarter', 'both_entity', 'leanings'])['sentistr_std'].mean().reset_index()
temp.to_csv('pos_ner_basic/leanings_np_both_entities_senti_std.csv', encoding='utf-8-sig')


temp  = pages_df_st.groupby(['quarter', 'out_group_np', 'leanings'])['insult_std'].mean().reset_index()
temp.to_csv('pos_ner_basic/leanings_np_out_group_insult_std.csv', encoding='utf-8-sig')


temp  = pages_df_st.groupby(['quarter', 'in_group_np', 'leanings'])['insult_std'].mean().reset_index()
temp.to_csv('pos_ner_basic/leanings_np_in_group_insult_std.csv', encoding='utf-8-sig')


temp  = pages_df_st.groupby(['quarter', 'no_entity_np', 'leanings'])['insult_std'].mean().reset_index()
temp.to_csv('pos_ner_basic/leanings_np_no_entities_insult_std.csv', encoding='utf-8-sig')

temp  = pages_df_st.groupby(['quarter', 'both_entity', 'leanings'])['insult_std'].mean().reset_index()
temp.to_csv('pos_ner_basic/leanings_np_both_entities_insult_std.csv', encoding='utf-8-sig')
